<a href="https://colab.research.google.com/github/kyle-gao/PySpark_Exercises/blob/master/PySpark_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 32.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=ccfd2087e4489853ac62cff0b1467b443c5f9c6eac32f000deb78070a2f40747
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

--2020-09-21 21:51:14--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/coursera_ml/a2.parquet [following]
--2020-09-21 21:51:15--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ml/a2.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ml/a2.parquet [following]
--2020-09-21 21:51:15--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP reque

In [7]:
df = spark.read.load('a2.parquet') #spark dataframe similar to pandas

In [10]:
df.head()

Row(CLASS=0, SENSORID=26, X=380.66434005495194, Y=-139.3470983812975, Z=-247.93697521077704)

In [14]:
df.createOrReplaceTempView("Accelerometer_Data") #allows for spark to call this Table using SQL

In [15]:
spark.sql("SELECT * from Accelerometer_Data").show() 

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

Decision Trees are usually the first supervsed algorithm to try. 

They work well out of the box and are good benchmarks for future models.

In [25]:
spark.sql("SELECT COUNT(DISTINCT CLASS) from Accelerometer_Data").show()#binary classification

+---------------------+
|count(DISTINCT CLASS)|
+---------------------+
|                    2|
+---------------------+



In [40]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

In [43]:
tree = DecisionTreeClassifier(labelCol="CLASS", featuresCol="features")
evaluator = MulticlassClassificationEvaluator(
    labelCol="CLASS", predictionCol="prediction", metricName="accuracy")

In [38]:
vecAssembler = VectorAssembler(inputCols=["X", "Y", "Z"], outputCol="features")
pipeline = Pipeline(stages=[vectorAssembler, tree])

In [68]:
model = pipeline.fit(trainingData)

In [69]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="CLASS", predictionCol="prediction", metricName="accuracy")
predictions = model.transform(testData)

In [70]:
predictions.select("X","Y","Z","prediction").show(10)

+-------------------+------------------+-------------------+----------+
|                  X|                 Y|                  Z|prediction|
+-------------------+------------------+-------------------+----------+
| 122.79284074820067|-88.19527091272191|-185.40334606851977|       0.0|
| 137.48274121419482| -93.9756133320119| -220.9712437858673|       0.0|
| -33.87457888688617|13.353554565156802| -443.7397084372543|       0.0|
|  144.8763646036816|-43.63767640465293|  36.56313967722145|       0.0|
|-56.735799011890066| 81.98728822873187| -17.89864645185649|       0.0|
|-4.8281721129789315|-67.38050508399905| 221.24876396496404|       0.0|
| 25.023569867449666|-53.21428926366043| -6.843856965266909|       0.0|
|-137.06896619507458|149.26988943191122|  -75.2732620551891|       0.0|
| -75.90246445601944|115.50570790739657| 111.11911236055724|       0.0|
| -56.73574214427017| 81.98727523527324| -17.89860240383493|       0.0|
+-------------------+------------------+-------------------+----

In [71]:
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % (accuracy)) #Wow the data must be really separable. 

Accuracy = 0.998927 
